# Required Imports

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import json
import re

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
df = pd.read_csv('dataset.csv')

In [5]:
# ! pip install datasets
# ! pip install transformers[torch]
# ! pip install tokenizers
# ! pip install evaluate
# ! pip install rouge_score
# ! pip install sentencepiece
# ! pip install huggingface_hub
# !pip install timexy

In [6]:
import nltk
import evaluate
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict
import timexy
from timexy import Timexy
from timexy import rule
from timexy.languages import en

2024-03-16 15:22:32.261551: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-16 15:22:32.263567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-16 15:22:32.305695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 15:22:33.171186: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
nlp = spacy.load("en_core_web_sm")

# Optionally add config if varying from default values
config = {
    "kb_id_type": "timex3",  # possible values: 'timex3'(default), 'timestamp'
    "label": "timexy",       # default: 'timexy'
    "overwrite": False       # default: False
}
nlp.add_pipe("timexy", config=config, before="ner")

# Preprocessing

In [38]:
def preprocess(batch):
    p,clean_batch = [],[]
    for i in batch['Inputs']:
        for j in i.split('\n')[:-1]:
            x = re.split('~__(True|False)__',j)
            #print(x)
            clean_batch.append(x[0])
            p.append(eval(x[1]))
    inputs = ["Generate : " + sentence for sentence in clean_batch]
    model_inputs = tokenizer(inputs,truncation=True)
    labels = tokenizer(text_target=list(batch['Outputs']),truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs,p

In [39]:
# train (1081)
c=0
for index, row in df.iterrows():
    if((row['Chunk'])<=1081):
        c+=1
train = df.iloc[:c,:]

# validate (rest)
test = df.iloc[c:,:]

# Training

In [40]:
# Run it the first time
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

chkpnt = './input/checkpoints/chkpnt1'
model.save_pretrained(chkpnt)
tokenizer.save_pretrained(chkpnt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./input/checkpoints/chkpnt1/tokenizer_config.json',
 './input/checkpoints/chkpnt1/special_tokens_map.json',
 './input/checkpoints/chkpnt1/spiece.model',
 './input/checkpoints/chkpnt1/added_tokens.json')

In [41]:
model = T5ForConditionalGeneration.from_pretrained(chkpnt)
tokenizer = T5Tokenizer.from_pretrained(chkpnt)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
tokenized_dataset,p = preprocess(train)
test_dataset,p1 = preprocess(test)

In [ ]:
# Training Arguments
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset,
   eval_dataset=test_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)